In [23]:
import numpy as np
import pandas as pd

\begin{align}
score_{new} = \color{orange}{max(scores_{new})} - \color{blue}{(max(scores_{new})-min(scores_{new}))} \cdot \color{red}{\frac{max(scores_{old})-score_{old}}{max(scores_{old})-min(scores_{old})}}
\end{align}

In [5]:
score_old_min=4
score_old_max=1
score_new_min=10
score_new_max=20

In [40]:
def scores_scale(X, scores_old_min, scores_old_max, scores_new_min=0, scores_new_max=1):
    X = scores_new_max - ((scores_new_max - scores_new_min) * (scores_old_max - X) / (scores_old_max - scores_old_min))
    return X

     0
20  20
16  16
12  12
10  10
      0
20  1.0
16  2.2
12  3.4
10  4.0


In [41]:
X = pd.DataFrame([[20, 16, 12, 10]]).T
X.index = X[0]
print(X)
print(scores_scale(X=X, scores_old_min=10, scores_old_max=20, scores_new_min=4, scores_new_max=1))

     0
0     
20  20
16  16
12  12
10  10
      0
0      
20  1.0
16  2.2
12  3.4
10  4.0


In [ ]:
class ScoreScaler(BaseEstimator, TransformerMixin):
    """Transforms features by scaling each feature to given scoring scale.

    This estimator scales and translates each feature individually such
    that it acccords with a given range on the training set, e.g. between
    zero and one. Without scale arguments, ScoreScaler acts like MinMaxScaler.

    Parameters
    ----------
    scores_old_min : int, float, or 'auto'; default 'auto'
        The smallest/worst score on the original scale. If 'auto', the smallest value of 
        each feature is assumed to be the smallest possible value.

    scores_old_max : int, float, or 'auto'; default 'auto'
        The highest/best score on the original scale. If 'auto', the greatest value of 
        each feature is assumed to be the highest possible value.
        
    scores_new_min : int or float; default 0
        The smallest/worst score on the transformed scale.
    
    scores_new_max : int or float; default 1
        The highest/best score on the transformed scale.

    Notes
    -----
    NaNs are treated as missing values: disregarded in fit, and maintained in
    transform.
    """

    def __init__(self, scores_old_min='auto', scores_old_max='auto', scores_new_min=0, scores_new_max=1):
        self.scores_old_min = scores_old_min
        self.scores_old_max = scores_old_max
        self.scores_new_min = scores_new_min
        self.scores_new_max = scores_new_max

    def fit(self, X, y=None):
        """Compute the minimum and maximum to be used for later scaling, if no score range is given.

        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data used to compute the per-feature minimum and maximum
            used for later scaling along the features axis.
        """
        
        if scores_old_min == 'auto':
            self.scores_old_min_ = X.min()
        if scores_old_max == 'auto':
            self.scores_old_max_ = X.max()
        return self

    def transform(self, X):
        """Scaling features of X according to scale settings.

        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            Input data that will be transformed.
        """
        
        X = self.scores_new_max - ((self.scores_new_max - self.scores_new_min) * 
                                   (self.scores_old_max_ - X) / (self.scores_old_max_ - self.scores_old_min_))
        return X

    def inverse_transform(self, X):
        """Undo the scaling of X according to scale settings.

        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            Input data that will be transformed.
        """

        X = self.scores_old_max_ - ((self.scores_old_max_ - self.scores_old_min_) * 
                                   (self.scores_new_max - X) / (self.scores_new_max - self.scores_new_min))
        return X